In [1]:
import pandas as pd

In [30]:
# Load fx data
data = pd.read_csv('C:/Users/Shree/PycharmProjects/FxPrediction/EUR_USD Historical Data.csv')
del data['Open']
del data['High']
del data['Low']
del data['Price']
#data = data.tail(60)

In [31]:
data['Change %'] = data['Change %'].str.replace('%', '').astype('float')

In [32]:
data

Date  Change %
0     May 24, 2019      0.21
1     May 23, 2019      0.24
2     May 22, 2019     -0.06
3     May 21, 2019     -0.10
4     May 20, 2019      0.12
5     May 17, 2019     -0.14
6     May 16, 2019     -0.25
7     May 15, 2019     -0.02
8     May 14, 2019     -0.18
9     May 13, 2019     -0.10
10    May 10, 2019      0.12
11    May 09, 2019      0.27
12    May 08, 2019      0.01
13    May 07, 2019     -0.08
14    May 06, 2019     -0.02
15    May 03, 2019      0.21
16    May 02, 2019     -0.15
17    May 01, 2019     -0.19
18    Apr 30, 2019      0.28
19    Apr 29, 2019      0.33
20    Apr 26, 2019      0.15
21    Apr 25, 2019     -0.20
22    Apr 24, 2019     -0.65
23    Apr 23, 2019     -0.28
24    Apr 22, 2019      0.09
25    Apr 19, 2019      0.15
26    Apr 18, 2019     -0.58
27    Apr 17, 2019      0.14
28    Apr 16, 2019     -0.25
29    Apr 15, 2019      0.06
...            ...       ...
1035  Jun 05, 2015     -1.12
1036  Jun 04, 2015     -0.32
1037  Jun 03, 2015      1.08
1038  Jun 02, 2015      2.08
1039  Jun 01, 2015     -0.55
1040  May 29, 2015      0.36
1041  May 28, 2015      0.40
1042  May 27, 2015      0.29
1043  May 26, 2015     -0.96
1044  May 25, 2015     -0.36
1045  May 22, 2015     -0.85
1046  May 21, 2015      0.18
1047  May 20, 2015     -0.50
1048  May 19, 2015     -1.47
1049  May 18, 2015     -1.20
1050  May 15, 2015      0.37
1051  May 14, 2015      0.47
1052  May 13, 2015      1.28
1053  May 12, 2015      0.53
1054  May 11, 2015     -0.44
1055  May 08, 2015     -0.56
1056  May 07, 2015     -0.76
1057  May 06, 2015      1.49
1058  May 05, 2015      0.36
1059  May 04, 2015     -0.50
1060  May 01, 2015     -0.20
1061  Apr 30, 2015      0.84
1062  Apr 29, 2015      1.38
1063  Apr 28, 2015      0.85
1064  Apr 27, 2015      0.13

[1065 rows x 2 columns]

In [45]:
#divide into train and validation set
train = data[:int(0.95*(len(data)))]
valid = data[int(0.95*(len(data))):]

In [46]:
#preprocessing (since arima takes univariate series as input)
train.drop('Date',axis=1,inplace=True)
valid.drop('Date',axis=1,inplace=True)

C:\Users\Shree\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [47]:
#plotting the data
import matplotlib.pyplot as plt
plt.plot(train['Change %'])
plt.plot(valid['Change %'])

In [48]:
#building the model
#from pyramid.arima import auto_arima
from pmdarima import auto_arima

In [49]:
#train

In [50]:
#building the model
from pmdarima import auto_arima

In [51]:
model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)

Fit ARIMA: order=(2, 0, 2) seasonal_order=(0, 0, 0, 1); AIC=1459.612, BIC=1489.124, Fit time=2.880 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=1455.729, BIC=1465.567, Fit time=0.199 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=1457.535, BIC=1472.291, Fit time=0.130 seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=1457.517, BIC=1472.273, Fit time=0.265 seconds
Fit ARIMA: order=(1, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=1456.074, BIC=1475.749, Fit time=1.530 seconds
Total fit time: 5.012 seconds


In [57]:
model_fit = model.fit(train)

In [58]:
print (model_fit.summary())

                           Statespace Model Results                           
Dep. Variable:                      y   No. Observations:                 1011
Model:                        SARIMAX   Log Likelihood                -725.865
Date:                Sun, 26 May 2019   AIC                           1455.729
Time:                        14:13:40   BIC                           1465.567
Sample:                             0   HQIC                          1459.466
                               - 1011                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0027      0.016      0.170      0.865      -0.028       0.033
sigma2         0.2461      0.007     34.064      0.000       0.232       0.260
Ljung-Box (Q):                       35.17   Jarque-

In [53]:
forecast = model.predict(n_periods=len(valid))

In [54]:
forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])

In [55]:
#plot the predictions for validation set
plt.plot(train, label='Train')
plt.plot(valid, label='Valid')
plt.plot(forecast, label='Prediction')
plt.show()

In [56]:
#calculate rmse
from math import sqrt
from sklearn.metrics import mean_squared_error

rms = sqrt(mean_squared_error(valid,forecast))
print(rms)

0.7661195955365451
